In [1]:
import linedraw

In [4]:
from random import *
import math
import argparse

from PIL import Image, ImageDraw, ImageOps

from filters import *
from strokesort import *
import perlin
from util import *

import numpy as np
import cv2

In [51]:
no_cv = False
no_svg = False
export_path = "output/out.svg"
draw_contours = True
draw_hatch = True
show_bitmap = False
resolution = 1024
hatch_size = 16
contour_simplify = 2

In [52]:
path = "images/lion.jpeg"
image = Image.open(path)
w, h = image.size

In [53]:
im_l = image.convert("L")
im_r, im_g, im_b = image.split()

im_l = ImageOps.autocontrast(im_l,10)
im_r = ImageOps.autocontrast(im_r,10)
im_g = ImageOps.autocontrast(im_g,10)
im_b = ImageOps.autocontrast(im_b,10)

im_channels = [(im_l, "black"), (im_r, "red"), (im_g, "green"), (im_b, "blue")]

In [54]:
lines_dict = {}
for im, color in im_channels:
    print(f'color: {color}')

    lines = []
    if draw_contours:
        lines += linedraw.getcontours(im.resize((resolution//contour_simplify,resolution//contour_simplify*h//w)),contour_simplify)
    if draw_hatch:
        lines += linedraw.hatch(im.resize((resolution//hatch_size,resolution//hatch_size*h//w)),hatch_size)

    lines = sortlines(lines)

    lines_dict[color] = lines

color: black
generating contours...
finding edges...
getting contour points...
connecting contour points...
getting contour points...
connecting contour points...
hatching...
optimizing stroke sequence...
color: red
generating contours...
finding edges...
getting contour points...
connecting contour points...
getting contour points...
connecting contour points...
hatching...
optimizing stroke sequence...
color: green
generating contours...
finding edges...
getting contour points...
connecting contour points...
getting contour points...
connecting contour points...
hatching...
optimizing stroke sequence...
color: blue
generating contours...
finding edges...
getting contour points...
connecting contour points...
getting contour points...
connecting contour points...
hatching...
optimizing stroke sequence...


In [55]:
def makesvg(lines):
    print("generating svg file...")
    out = '<svg xmlns="http://www.w3.org/2000/svg" version="1.1">'
    if (type(lines) == dict):
        for color in lines:
            for l in lines[color]:
                l = ",".join([str(p[0]*0.5)+","+str(p[1]*0.5) for p in l])
                out += '<polyline points="'+l+f'" stroke="{color}" stroke-width="2" fill="none" />\n'
        
    else:
        for l in lines:
            l = ",".join([str(p[0]*0.5)+","+str(p[1]*0.5) for p in l])
            out += '<polyline points="'+l+'" stroke="black" stroke-width="2" fill="none" />\n'
    
    out += '</svg>'
    return out

In [56]:
out_svg = makesvg(lines_dict)

generating svg file...


In [57]:
f = open(export_path,'w')
f.write(out_svg)
f.close()

In [58]:
disp = Image.new("RGB",(resolution,resolution*h//w),(255,255,255))
draw = ImageDraw.Draw(disp)
if (type(lines_dict) == dict):
    for color in lines:
        for l in lines:
            if color == 'black':
                draw.line(l,(0,0,0),5)
            elif color == 'red':
                draw.line(l,(255,0,0),5)
            elif color == 'green':
                draw.line(l,(0,255,0),5)
            elif color == 'blue':
                draw.line(l,(0,0,255),5)
else:
    for l in lines:
        draw.line(l,(0,0,0),5)
disp.show()

/snap/core20/current/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.29' not found (required by /lib/x86_64-linux-gnu/libproxy.so.1)
Failed to load module: /home/babos/snap/code/common/.cache/gio-modules/libgiolibproxy.so
eog: symbol lookup error: /snap/core20/current/lib/x86_64-linux-gnu/libpthread.so.0: undefined symbol: __libc_pthread_init, version GLIBC_PRIVATE
